In [1]:
import data_handler_uniNER

In [2]:
uniNER_dataset_MSEQA_format = data_handler_uniNER.build_uniNER_dataset_MSEQA_format()

Using custom data configuration Universal-NER--Pile-NER-type-e81e7ad3a512a503
Found cached dataset json (/Users/andrew/.cache/huggingface/datasets/Universal-NER___json/Universal-NER--Pile-NER-type-e81e7ad3a512a503/0.0.0/e6070c77f18f01a5ad4551a8b7edfba20b8438b7cad4d94e6ad9378022ce4aab)


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
print(uniNER_dataset_MSEQA_format['train'][23])

In [5]:
from transformers import AutoTokenizer
import transformers

In [6]:
tokenizer_to_use = "roberta-base"

In [7]:
MAX_SEQ_LENGTH = 512  # question + context + special tokens
DOC_STRIDE = 64  # overlap between 2 consecutive passages from same document
MAX_QUERY_LENGTH = 48  # not used, but questions must not be too long given a chosen DOC_STRIDE

In [9]:
print("\nLoading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(tokenizer_to_use)
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast)
MODEL_CONTEXT_WINDOW = tokenizer.model_max_length
assert MAX_SEQ_LENGTH <= MODEL_CONTEXT_WINDOW, "MAX SEQ LENGTH must be smallerEqual than model context window"
print(f"MAX_SEQ_LENGTH used to chunk documents: {MAX_SEQ_LENGTH}")
assert DOC_STRIDE < (MAX_SEQ_LENGTH - MAX_QUERY_LENGTH), "DOC_STRIDE must be smaller, otherwise parts of the doc will be skipped"
print("DOC_STRIDE used: {}".format(DOC_STRIDE))


Loading tokenizer...
MAX_SEQ_LENGTH used to chunk documents: 512
DOC_STRIDE used: 64


In [10]:
prefix = "Your task is to extract the Named Entities of type PROCESS from an input TEXT. You are given a DEFINITION and some GUIDELINES.\nDEFINITION: PROCESS refers to a series of actions, changes, or functions that lead to a particular result or outcome, such as procedures, methodologies, and workflows.\nGUIDELINES: Do not label general activities or events without a clear sequence. Exercise caution with ambiguous terms like 'production' (could refer to a process, a company, or a location) and 'assembly' (could refer to a process or a gathering of people).\nTEXT:"

In [11]:
print(prefix)

Your task is to extract the Named Entities of type PROCESS from an input TEXT. You are given a DEFINITION and some GUIDELINES.
DEFINITION: PROCESS refers to a series of actions, changes, or functions that lead to a particular result or outcome, such as procedures, methodologies, and workflows.
GUIDELINES: Do not label general activities or events without a clear sequence. Exercise caution with ambiguous terms like 'production' (could refer to a process, a company, or a location) and 'assembly' (could refer to a process or a gathering of people).
TEXT:


In [12]:
tokenized = tokenizer(prefix,
        truncation='only_second',  # longest_first
        max_length=MAX_SEQ_LENGTH,
        stride=DOC_STRIDE,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding=False,  # not padding here
    )

In [14]:
print(len(tokenized['input_ids'][0]))

134
